# flow collecting data

In [ ]:
import cv2
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def mouse_callback(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(params['points']) < 4:
            params['points'].append((x, y))
            cv2.circle(params['img'], (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Image-pick point clockwise", params['img'])

def transform_image(input_image, points):
    width, height = [ max([points[0][0],points[1][0],points[2][0],points[3][0]])-min([points[0][0],points[1][0],points[2][0],points[3][0]]) , max([points[0][1],points[1][1],points[2][1],points[3][1]])-min([points[0][1],points[1][1],points[2][1],points[3][1]])]
    pts1 = np.float32(points)
    pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    result0 = cv2.warpPerspective(input_image, matrix, (width, height))
    return [result0,[width, height]]
    
def extract_frames_collecting_flowdata(video_path, output_folder, interval=1):
    pickpointstatus = False
    flowratelist=[]
    
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    print ("Actual frame rate: ",fps)
    fps = math.ceil(fps)
    print ("Calculate fps:", fps)
    while True:
        ret, frame = cap.read()
        
        ### rotatio ####################################################
        # frame= cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        if not ret:
            break

        frame_count += 1
        #print ("current frame", frame_count)

        # Calculate time in seconds
        current_time = frame_count / fps
        #print (current_time)
        # Check if the current time is a multiple of the interval
        if frame_count % fps == 0 :
            # Save the frame
            output_path = os.path.join(output_folder, f"frame_{frame_count}_transformted.jpg")
            
            input_image = frame
            if pickpointstatus == False:
                # Create a copy of the input image for drawing points
                img_copy = input_image.copy()
                
                # Store the points
                points = []   
                # Set mouse callback function
                cv2.namedWindow("Image-pick point clockwise")
                cv2.setMouseCallback("Image-pick point clockwise", mouse_callback, {'img': img_copy, 'points': points})
                
                # Display the input image
                cv2.imshow("Image-pick point clockwise", img_copy)
                cv2.waitKey(0)
                print("pick point clockwise")
                print(points)
            result = transform_image(input_image, points)
            if pickpointstatus == False:
                cv2.imshow("Transformed Image", result[0])
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                print("domain pixel w-h",result[1])
                print("domain pixel ratio",result[1][0]/result[1][1])
                pickpointstatus = True
            # cv2.imwrite(output_path, result[0])
            cv2.imshow("",result[0])
            cv2.moveWindow("", 650, 50) 
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            print("current time =", current_time, " flow late =", end="")
            flowrate= str(input())
            if flowrate!="":
                flowrate=int(flowrate)/10
            flowratelist.append([current_time, flowrate])

    plt.show()
    flowratelist=np.array(flowratelist)
    # flowratelist[:,1:] = flowratelist[:,1:]*-100
    df = pd.DataFrame(flowratelist, columns=['time (s)', 'flowlate (L/min)'])
    df.to_csv(output_folder+'/finepixel '+testNo+' flow.csv', index=False)
    df.plot(x="time (s)", y='flowlate (L/min)')
    print("RGB values saved")       

    # Release video capture object
    cap.release()
    print("total frames= ", frame_count )
    print(f"Frames extracted at {interval} second interval and saved to {output_folder}")


# input
test=5
repeat=2

testNo = "test"+str(test)+"."+str(repeat)
flowtestNo = "flow"+str(test)+"."+str(repeat)
folder=["","test1 V0.65 F15","test2 V0.65 F25","test3 V0.65 F35","test4 V0.7 F35","test5 V0.60 F35"]
video_path = "./asset/"+folder[test]+"/"+testNo+"/"+flowtestNo+".MOV"
output_folder = "./asset/"+folder[test]+"/"+testNo
interval = 1  # default Extract frames every 1 second

extract_frames_collecting_flowdata(video_path, output_folder, interval)
